In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

NameError: name 'nn' is not defined

In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3005.054199
Minibatch accuracy: 7.0%
Validation accuracy: 14.6%
Minibatch loss at step 500: 2.528866
Minibatch accuracy: 49.2%
Validation accuracy: 48.2%
Minibatch loss at step 1000: 2.133094
Minibatch accuracy: 52.3%
Validation accuracy: 50.7%
Minibatch loss at step 1500: 2.360604
Minibatch accuracy: 40.6%
Validation accuracy: 47.5%
Minibatch loss at step 2000: 2.320079
Minibatch accuracy: 46.9%
Validation accuracy: 54.5%
Minibatch loss at step 2500: 2.190601
Minibatch accuracy: 46.9%
Validation accuracy: 49.2%
Minibatch loss at step 3000: 2.215058
Minibatch accuracy: 53.1%
Validation accuracy: 45.3%
Test accuracy: 48.4%


In [8]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3071.965576
Minibatch accuracy: 10.9%
Validation accuracy: 14.7%
Minibatch loss at step 500: 2.528866
Minibatch accuracy: 49.2%
Validation accuracy: 48.2%
Minibatch loss at step 1000: 2.133093
Minibatch accuracy: 52.3%
Validation accuracy: 50.7%
Minibatch loss at step 1500: 2.360604
Minibatch accuracy: 40.6%
Validation accuracy: 47.5%
Minibatch loss at step 2000: 2.320079
Minibatch accuracy: 46.9%
Validation accuracy: 54.5%
Minibatch loss at step 2500: 2.190601
Minibatch accuracy: 46.9%
Validation accuracy: 49.2%
Minibatch loss at step 3000: 2.215058
Minibatch accuracy: 53.1%
Validation accuracy: 45.3%
Test accuracy: 48.4%


In [10]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + tf.nn.l2_loss(weights)+ tf.nn.l2_loss(biases)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3032.094727
Minibatch accuracy: 10.9%
Validation accuracy: 19.0%
Minibatch loss at step 500: 2.804839
Minibatch accuracy: 46.1%
Validation accuracy: 47.0%
Minibatch loss at step 1000: 2.712131
Minibatch accuracy: 44.5%
Validation accuracy: 45.6%
Minibatch loss at step 1500: 2.457960
Minibatch accuracy: 42.2%
Validation accuracy: 41.9%
Minibatch loss at step 2000: 2.449770
Minibatch accuracy: 44.5%
Validation accuracy: 52.3%
Minibatch loss at step 2500: 2.299901
Minibatch accuracy: 46.1%
Validation accuracy: 48.1%
Minibatch loss at step 3000: 2.460700
Minibatch accuracy: 43.8%
Validation accuracy: 45.9%
Test accuracy: 49.7%


In [12]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3093.417236
Minibatch accuracy: 6.2%
Validation accuracy: 19.5%
Minibatch loss at step 500: 2.804839
Minibatch accuracy: 46.1%
Validation accuracy: 47.0%
Minibatch loss at step 1000: 2.712131
Minibatch accuracy: 44.5%
Validation accuracy: 45.6%
Minibatch loss at step 1500: 2.457960
Minibatch accuracy: 42.2%
Validation accuracy: 41.9%
Minibatch loss at step 2000: 2.449770
Minibatch accuracy: 44.5%
Validation accuracy: 52.3%
Minibatch loss at step 2500: 2.299901
Minibatch accuracy: 46.1%
Validation accuracy: 48.1%
Minibatch loss at step 3000: 2.460700
Minibatch accuracy: 43.8%
Validation accuracy: 45.9%
Minibatch loss at step 3500: 2.513746
Minibatch accuracy: 36.7%
Validation accuracy: 41.9%
Minibatch loss at step 4000: 2.540963
Minibatch accuracy: 38.3%
Validation accuracy: 44.2%
Minibatch loss at step 4500: 2.248144
Minibatch accuracy: 47.7%
Validation accuracy: 43.1%
Minibatch loss at step 5000: 2.026199
Minibatch accuracy: 57.8%
Validation accur

In [13]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 5e-4*(tf.nn.l2_loss(weights)+ tf.nn.l2_loss(biases))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [14]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.474157
Minibatch accuracy: 12.5%
Validation accuracy: 14.6%
Minibatch loss at step 500: 2.054069
Minibatch accuracy: 82.8%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 1.458362
Minibatch accuracy: 78.1%
Validation accuracy: 77.5%
Minibatch loss at step 1500: 1.617522
Minibatch accuracy: 78.9%
Validation accuracy: 78.7%
Minibatch loss at step 2000: 1.111915
Minibatch accuracy: 81.2%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 0.900709
Minibatch accuracy: 84.4%
Validation accuracy: 80.7%
Minibatch loss at step 3000: 0.859529
Minibatch accuracy: 82.0%
Validation accuracy: 80.0%
Test accuracy: 86.7%


In [15]:
batch_size = 128
hnnum = 2048

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hnnum]))
  biases_1 = tf.Variable(tf.zeros([hnnum]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hnnum, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  reluout = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(reluout, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2, tf_train_labels))
  loss += 5e-4*(tf.nn.l2_loss(weights_1)+ tf.nn.l2_loss(biases_1) + 
               tf.nn.l2_loss(weights_2)+ tf.nn.l2_loss(biases_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  vlogits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  vreluout = tf.nn.relu(vlogits_1)
  vlogits_2 = tf.matmul(vreluout, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(vlogits_2)
  tlogits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  treluout = tf.nn.relu(tlogits_1)
  tlogits_2 = tf.matmul(treluout, weights_2) + biases_2
  test_prediction = tf.nn.softmax(tlogits_2)

In [16]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 813.420532
Minibatch accuracy: 3.1%
Validation accuracy: 19.1%
Minibatch loss at step 500: 255.568298
Minibatch accuracy: 86.7%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 192.960495
Minibatch accuracy: 85.9%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 151.414246
Minibatch accuracy: 82.8%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 115.102997
Minibatch accuracy: 88.3%
Validation accuracy: 84.1%
Minibatch loss at step 2500: 90.161720
Minibatch accuracy: 82.8%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 71.161278
Minibatch accuracy: 87.5%
Validation accuracy: 84.3%
Test accuracy: 90.5%


In [17]:
batch_size = 128
hnnum = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hnnum]))
  biases_1 = tf.Variable(tf.zeros([hnnum]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hnnum, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  reluout = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(reluout, weights_2) + biases_2
  logits_2_dp = tf.nn.dropout(logits_2, 0.5, seed=SEED)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2_dp, tf_train_labels))
  loss += 5e-4*(tf.nn.l2_loss(weights_1)+ tf.nn.l2_loss(biases_1) + 
               tf.nn.l2_loss(weights_2)+ tf.nn.l2_loss(biases_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  vlogits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  vreluout = tf.nn.relu(vlogits_1)
  vlogits_2 = tf.matmul(vreluout, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(vlogits_2)
  tlogits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  treluout = tf.nn.relu(tlogits_1)
  tlogits_2 = tf.matmul(treluout, weights_2) + biases_2
  test_prediction = tf.nn.softmax(tlogits_2)

NameError: name 'SEED' is not defined

In [18]:
batch_size = 128
hnnum = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hnnum]))
  biases_1 = tf.Variable(tf.zeros([hnnum]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hnnum, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  reluout = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(reluout, weights_2) + biases_2
  logits_2_dp = tf.nn.dropout(logits_2, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2_dp, tf_train_labels))
  loss += 5e-4*(tf.nn.l2_loss(weights_1)+ tf.nn.l2_loss(biases_1) + 
               tf.nn.l2_loss(weights_2)+ tf.nn.l2_loss(biases_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  vlogits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  vreluout = tf.nn.relu(vlogits_1)
  vlogits_2 = tf.matmul(vreluout, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(vlogits_2)
  tlogits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  treluout = tf.nn.relu(tlogits_1)
  tlogits_2 = tf.matmul(treluout, weights_2) + biases_2
  test_prediction = tf.nn.softmax(tlogits_2)

In [19]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 639.975342
Minibatch accuracy: 6.2%
Validation accuracy: 23.0%
Minibatch loss at step 500: 139.739639
Minibatch accuracy: 85.2%
Validation accuracy: 79.1%
Minibatch loss at step 1000: 107.929298
Minibatch accuracy: 82.8%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 76.842766
Minibatch accuracy: 74.2%
Validation accuracy: 79.6%
Minibatch loss at step 2000: 59.365192
Minibatch accuracy: 86.7%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 46.363209
Minibatch accuracy: 83.6%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 36.264446
Minibatch accuracy: 79.7%
Validation accuracy: 82.1%
Test accuracy: 88.5%


In [20]:
batch_size = 128
hnnum = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hnnum]))
  biases_1 = tf.Variable(tf.zeros([hnnum]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hnnum, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  reluout = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(reluout, weights_2) + biases_2
  logits_2_dp = tf.nn.dropout(logits_2, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2_dp, tf_train_labels))
  loss += 5e-4*(tf.nn.l2_loss(weights_1)+ tf.nn.l2_loss(biases_1) + 
               tf.nn.l2_loss(weights_2)+ tf.nn.l2_loss(biases_2))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  vlogits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  vreluout = tf.nn.relu(vlogits_1)
  vlogits_2 = tf.matmul(vreluout, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(vlogits_2)
  tlogits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  treluout = tf.nn.relu(tlogits_1)
  tlogits_2 = tf.matmul(treluout, weights_2) + biases_2
  test_prediction = tf.nn.softmax(tlogits_2)

TypeError: exponential_decay() missing 1 required positional argument: 'decay_rate'

In [21]:
batch_size = 128
hnnum = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hnnum]))
  biases_1 = tf.Variable(tf.zeros([hnnum]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hnnum, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  reluout = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(reluout, weights_2) + biases_2
  logits_2_dp = tf.nn.dropout(logits_2, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2_dp, tf_train_labels))
  loss += 5e-4*(tf.nn.l2_loss(weights_1)+ tf.nn.l2_loss(biases_1) + 
               tf.nn.l2_loss(weights_2)+ tf.nn.l2_loss(biases_2))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 0.95)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  vlogits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  vreluout = tf.nn.relu(vlogits_1)
  vlogits_2 = tf.matmul(vreluout, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(vlogits_2)
  tlogits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  treluout = tf.nn.relu(tlogits_1)
  tlogits_2 = tf.matmul(treluout, weights_2) + biases_2
  test_prediction = tf.nn.softmax(tlogits_2)

TypeError: exponential_decay() missing 1 required positional argument: 'decay_rate'

In [22]:
batch_size = 128
hnnum = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hnnum]))
  biases_1 = tf.Variable(tf.zeros([hnnum]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hnnum, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  reluout = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(reluout, weights_2) + biases_2
  logits_2_dp = tf.nn.dropout(logits_2, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2_dp, tf_train_labels))
  loss += 5e-4*(tf.nn.l2_loss(weights_1)+ tf.nn.l2_loss(biases_1) + 
               tf.nn.l2_loss(weights_2)+ tf.nn.l2_loss(biases_2))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(
      0.5,                # Base learning rate.
      global_step,  # Current index into the dataset.
      num_labels,          # Decay step.
      0.95,                # Decay rate.
      staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  vlogits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  vreluout = tf.nn.relu(vlogits_1)
  vlogits_2 = tf.matmul(vreluout, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(vlogits_2)
  tlogits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  treluout = tf.nn.relu(tlogits_1)
  tlogits_2 = tf.matmul(treluout, weights_2) + biases_2
  test_prediction = tf.nn.softmax(tlogits_2)

In [23]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 651.842529
Minibatch accuracy: 5.5%
Validation accuracy: 22.7%
Minibatch loss at step 500: 152.436981
Minibatch accuracy: 85.2%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 148.762634
Minibatch accuracy: 88.3%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 152.297470
Minibatch accuracy: 82.8%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 159.051620
Minibatch accuracy: 84.4%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 154.714355
Minibatch accuracy: 82.0%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 156.330322
Minibatch accuracy: 82.8%
Validation accuracy: 81.1%
Test accuracy: 87.8%
